In [27]:
import json
import csv
import requests
from sets import Set
import numpy as np
import cPickle
import pandas as pd
import sys
import time
import random

#### pair_list_5-6-7_2018.json and taskid_list_5-6-7-2018.txt are created by get_merged_tasks.py

In [2]:
with open('pair_list_5-6-7_2018.json') as json_file:  
    data = json.load(json_file)

In [34]:
# create set_list from duplicate key value pair
## remove the pair that contains ticket outside snaptask 5-7 2018 (data.keys())
set_list = []
for k in data.keys():
    for i in data[k]:
        if i in data.keys():
            set_list.append(Set([k,i]))

# tickets outside snaptask
m_list = []
for k in data.keys():
    for i in data[k]:
        if i not in data.keys():
            m_list.append(i)

In [35]:
# if keyword 'snaptask' is in description, we think the task is created by snaptask
def taskid_to_issnap(taskid):
    datat = [
              ('api.token', 'cli-XXX'),
              ('task_id', taskid),
            ]
    response = requests.post('https://code.uberinternal.com/api/maniphest.info', data=datat)
    jresponse = json.loads(response.text)
    return 'snaptask' in jresponse['result']['description']

In [36]:
# find tickets not from snaptask 5-7 2018
## (1) manually created master tasks
## (2) snaptask tickets created outside this time range 
manual_list = []
snap_list = []
for i in set(m_list):
    js = taskid_to_issnap(i)
    if js is True:
        snap_list.append(i)
    else:
        manual_list.append(i)

In [37]:
# for each m_master, get the snap tickets that linked with them
ct_manual = []
for m in set(manual_list):
    merged = [key for key, value in data.iteritems() if m in value]
    ct_manual.append(len(merged)-1)
    
for m in set(manual_list):
    merged = [key for key, value in data.iteritems() if m in value]
    merged_key = merged[0]
    for i in range(len(merged)-1):
        set_list.append(Set([merged_key,merged[i+1]]))

In [16]:
# for each snap ticket outside date range, get the snap tickets that linked with them
ct_snap = []
for m in set(snap_list):
    merged = [key for key, value in data.iteritems() if m in value]
    ct_snap.append(len(merged))

for m in set(snap_list):
    merged = [key for key, value in data.iteritems() if m in value]
    for i in merged:
        set_list.append(Set([m , i]))

In [17]:
pair_training = []
for l in Set(set_list):
    lst = list(l)
    pair_training.append(lst)

In [18]:
cPickle.dump(pair_training, open('5-6-7_2018_duplicated_pairs.p', 'wb')) 

---

In [20]:
dup_pairs = cPickle.load(open('5-6-7_2018_duplicated_pairs.p', 'rb'))

In [21]:
df_dup_pairs = pd.DataFrame.from_records(dup_pairs, columns=['task1','task2'])

In [29]:
df_snaptask_ids = pd.read_csv('taskid_list_5-6-7-2018.txt', sep=" ", header=None)
df_snaptask_ids.columns = ["taskid"]
list_snaptaskids = [i[1:] for i in df_snaptask_ids.taskid.values.tolist()]
df_dup_pairs = df_dup_pairs[(df_dup_pairs.task1.isin(list_snaptaskids) & df_dup_pairs.task2.isin(list_snaptaskids) )]

In [25]:
with open('5-6-7_matched_list.txt', 'a') as the_file:
    for i,j in zip(df_dup_pairs.task1.values.tolist(),df_dup_pairs.task2.values.tolist() ) :
        the_file.write(i+' '+j+'\n')

In [28]:
df = df_dup_pairs
d1 = df['task1'].tolist()
d2 = df['task2'].tolist()
random_list = []
for i in range(5000):
    p = random.sample(set(d1+d2), 2)
    if (df[(df['task1']==p[0]) & (df['task2']==p[1])].empty) \
       and (df[(df['task2']==p[0]) & (df['task1']==p[1])].empty):
        random_list.append(p)
df_random = pd.DataFrame.from_records(random_list, columns=['task1','task2'])   

In [30]:
df1lst = df_random.task1.values.tolist()
df2lst = df_random.task2.values.tolist()

In [31]:
with open('5-6-7_unmatched_list.txt', 'a') as the_file:
    for i,j in zip(df1lst, df2lst):
        the_file.write(i+' '+j+'\n')